In [187]:
import pandas as pd
import uuid

In [188]:
!pip install openpyxl


[notice] A new release of pip available: 22.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [189]:
#Crear dimensiones base para variable a nivel sujeto
temaInteres = pd.DataFrame(columns=['id','nombre','descripcion'])

etapa= pd.DataFrame(columns=['id','numEtapa','nombreAnalisis','nombreBD','descripcion',
                             'eventoInicial','eventoFinal'])

fecha=pd.DataFrame(columns=['dia','mes','anio'])

evento=pd.DataFrame(columns=['id','nombre','descripcion'])

variable=pd.DataFrame(columns=['id','nombreBD','nombreAnalisis','unidades','edadCorregida','derivada',
                               'impacto','tipoDato','tipoVariable','nivelMedicion','longitudinal',
                               'descripcionCorta','descripcionLarga'])


In [190]:
# Conseguir dataframes de diccionario
file_dict=pd.ExcelFile('data/KMC-diccionario.xlsx')
df_etapas=pd.read_excel(file_dict, sheet_name='ETAPAS')
df_temas=pd.read_excel(file_dict, sheet_name='Temas de Interés')
df_variables=pd.read_excel(file_dict, sheet_name='KMC-12mEC-DIC-VAR-version2024')
df_variables = df_variables.dropna(subset=['VAR-ID'])

In [191]:
#Poblar dimensión de eventos
eventos = pd.unique(
    pd.concat([
        df_etapas['Evento inicial (id-var)'],
        df_etapas['Evento-final (id-var)']
    ])
).tolist()

ids_eventos = [str(uuid.uuid4()) for _ in range(len(eventos))]

for i in range(len(eventos)):
    evento.loc[len(evento)] = [
        ids_eventos[i],
        eventos[i],
        ""
    ]

In [192]:
#Poblar dimensión de etapas
def encontrar_evento(nombre):
    fila=evento[evento['nombre']==nombre]
    return fila['id'].values[0]

ids_etapas = [str(uuid.uuid4()) for _ in range(len(df_etapas))]

for index, row in df_etapas.iterrows():
    etapa.loc[row['NUMERO ETAPA']]=[
        ids_etapas[index],
        row['NUMERO ETAPA'],
        row['ID-ETAPA'].lower(),
        row['NOMBRE CORTO'].lower(),
        row['DESCRIPCION'].lower(),
        encontrar_evento(row['Evento inicial (id-var)']),
        encontrar_evento(row['Evento-final (id-var)'])
    ]

In [193]:
#Poblar dimensión tema de interes
ids_temas = [str(uuid.uuid4()) for _ in range(len(df_temas))]

for index,row in df_temas.iterrows():
    temaInteres.loc[row['Número TdI']]=[
        ids_temas[index],
        row['ID-TdI'],
        row['DescripciOn']
    ]

In [194]:
#Poblar dimension fecha
fecha.loc[0]=[
    1,
    1,
    2025
]

In [195]:
#Poblar dimension variable

ids_vars = [str(uuid.uuid4()) for _ in range(len(df_variables))]

for index,row in df_variables.iterrows():
    if (index<634):
        variable.loc[index]=[
            ids_vars[index],
            row['NOMBRE EN LA BdeD'],
            row['VAR-ID'],
            row['UNITS'],
            False, #TODO: Edad coregida:De donde se saca esto?
            False, #TODO: Derivada: De donde se saca esto?
            False, #TODO: Impacto: ¿?
            row['VAR-TYPE'],
            False, #Tipo Variable
            False, #Nivel medicion
            False, #Longitudinal
            row['VAR-SHORT DESCRIPTION'],
            row['VAR-LONG DESCRIPTION']
        ]

In [196]:
#Crear diccionario de valor categorias
cols = [
    "VAL1", "SHORT-NAME1", "VAL2", "SHORT-NAME2",
    "VAL3", "SHORT-NAME3", "VAL4", "SHORT-NAME4", "VAL5", "SHORT-NAME5",
    "VAL6", "SHORT-NAME6", "VAL7", "SHORT-NAME7", "VAL8", "SHORT-NAME8",
    "VAL9", "SHORT-NAME9", "VAL10", "SHORT-NAME10", "VAL11", "SHORT-NAME11"
]

df_categorias=df_variables[cols]

unique_dicts = set()

def add_unique_dict(d):
    frozen = frozenset(d.items())
    if frozen not in unique_dicts:
        unique_dicts.add(frozen)

for index, row in df_categorias.iterrows():
    for i, col in  enumerate(df_categorias.columns):
        
        if i + 1 < len(df_categorias.columns):  
            siguiente_col = df_categorias.columns[i + 1]
        if pd.notna(row[col]) and 'VAL' in col and row[col] != ' ':
            dicc={
                'value':int(row[col]),
                'description':str(row[siguiente_col]).strip()
            }
            add_unique_dict(dicc)


In [197]:
valorCategoria=pd.DataFrame(columns=['id','codificacion','descripcion'])
for d in unique_dicts:
    data = dict(d) 
    nueva_fila = {
        'id': str(uuid.uuid4()),               
        'codificacion': data['value'],         
        'descripcion': data['description']     
    }
    valorCategoria.loc[len(valorCategoria)] = nueva_fila

valorCategoria

,id,codificacion,descripcion
0,8272c65f-c985-4ae2-befe-2ce7594b56a5,2,Problemas económicos
1,27a8ee29-6bcf-429d-a45f-5c767886ce01,2,AL MENOS 12 H
2,9d75e417-b128-4e4b-9307-8f758e54eed1,5,anticoagulante (sin antiHTA o antibio o B+)
3,2874579f-7a77-48ba-a520-50faf7ea06d7,1,de 40sem a 12 meses
4,27fcafc2-b0dc-4d7e-bf69-63596fafe75d,2,no deserto por ahora
...,...,...,...
311,223f8c41-3878-4e2c-883c-c005f770a7ec,1,Ventilación sola
312,e2be4e65-bff4-4b3d-8efa-4491b759ccaf,1,Med.Obstetra
313,0bcb51fb-8502-4845-ba4a-e39c2d656d1b,0,NO HIC
314,81a40eb3-2df8-42c5-8adb-b724b4db4dee,1,Cefálica


In [198]:
#cargar archivo de variables longitudinales
file_dict=pd.ExcelFile('data/KMC-varLongitudinal.xlsx')
df_vLongitudinales=pd.read_excel(file_dict, sheet_name='VLs-Definition')

In [199]:
df_vLongitudinales.rename(columns={'Unnamed: 0': 'descripcion'}, inplace=True)

In [200]:
df_vLongitudinales

,descripcion,ERN,ENTRADA PosCang,40S,3M,6M,9M,12M
0,VL_Weight,ERN_Peso,V196A,V218,V261,V304,V347,V389
1,VL_Height,ERN_Talla,V196B,V219,V262,V305,V348,V390
2,VL_HeadCircunference,ERN_PC,V196C,V220,V263,V306,V349,V391
3,VL_WeightForHeight,pesotalla0,BMI1,BMI2,NaN,NaN,NaN,BMI12
4,VL_zscoreWeight,zscorepeso0,zscorepeso1,zscorepeso2,zscorepeso3,zscorepeso6,zscorepeso9,zscorepeso12
5,VL_zscoreHeight,zscoretalla0,zscoretalla1,zscoretalla2,zscoretalla3,zscoretalla6,zscoretalla9,zscoretalla12
6,VL_zscoreHeadCircunf,zscorePC0,zscorePC1,zscorePC2,zscorePC3,zscorePC6,zscorePC9,zscorePC12
7,VL_zscoreWeightForHeight,zscorepesotalla0,zscorepesotalla1,zscorepesotalla2,zscorepesotalla3,zscorepesotalla6,zscorepesotalla9,zscorepesotalla12
8,VL_Nutrition,-,alisalida,ali40,ali3m,ali6m,ali9m,ali12m
9,VL-SpeedZscoreWeight,-,-,-,velocidadzscore3m_40semOMS,velocidad6_3mesesOMS,velocidad9_6mesesOMS,velocidad12_9mesesOMS


In [201]:
#Creación bridge y grupo variable longitudial
bridgeVaribleLongitudinal= pd.DataFrame(columns=['idGrupoVariable','idVariableCompuesta','descripcion'])
grupoVariableLongitudinal = pd.DataFrame(columns=['idGrupo'])

In [202]:
#Poblar grupo y bridge variable longitudinal
ids_grupoVLs = [str(uuid.uuid4()) for _ in range(10)]

cont=0
for index, row in df_vLongitudinales.iterrows():
    for col in df_vLongitudinales.columns:
        if col!='descripcion':
            nombre_variable = row[col]  # Valor específico en esa celda
            var = variable[variable['nombreBD'] == nombre_variable]
            if not var.empty:
                id_variable = var.iloc[0]['id']
                bridgeVaribleLongitudinal.loc[len(bridgeVaribleLongitudinal)]=[
                ids_grupoVLs[cont],
                id_variable,
                str(row['descripcion'])+'-'+str(col)
            ]
            
    cont+=1

for i in range (10):
    grupoVariableLongitudinal.loc[len(grupoVariableLongitudinal)]=[
        ids_grupoVLs[i]
    ]

In [203]:
#Creacion tabla de hechos variable sujeto 
hechoEvaluarVariableSujeto= pd.DataFrame(columns=['idVariable','idEtapa','idGrupoValorCategoria',
                                                  'idGrupoTemaInteres','idFechaModificacion','idFechaRegistro',
                                                  'idGrupoOperacion','idGrupoVariableLongitudinal'])

In [204]:
#Retorna la lista de ids de categorias relacionadas a  una fila (variable)
def definirGrupoIdValorCategoria(row):
    row=row[cols] #solo se tiene en cuenta las columnas de la codificación de categoria
    id=[]
    for i, col in  enumerate(df_categorias.columns):           
        if i + 1 < len(df_categorias.columns):  
            siguiente_col = df_categorias.columns[i + 1]
        if pd.notna(row[col]) and 'VAL' in col and row[col] != ' ':
                val=valorCategoria[
                    (valorCategoria['codificacion']==row[col]) &
                    (valorCategoria['descripcion']==row[siguiente_col])
                ]
                if not val.empty:
                    id.append(str(val['id'].iloc[0]))
                
    return id

In [205]:
#Creacion bridge tema de interes
bridgeTemaInteres= pd.DataFrame(columns=['idGrupoTemaInteres','idTemaInteres'])

In [206]:
#Poblar tabla de hechos 
bridgeCategoria=pd.DataFrame(columns=['idGrupoValorCategoria','idValorCategoria'])

ids_grupoCats = [str(uuid.uuid4()) for _ in range(len(variable))]
ids_grupoTemas= [str(uuid.uuid4()) for _ in range(len(variable))]

for index,row in df_variables.iterrows():
    #id variable
    row_var = variable[
        variable['nombreAnalisis']==
        str(row['VAR-ID'])
    ]  
    var= str(row_var['id'].iloc[0])
    
    #id etapa
    row_etapa=etapa[etapa['numEtapa']==(row['#ETAPA'])]
    id_etapa=str(row_etapa['id'].iloc[0])

    #id categoria y grupo
    grupo_categoria=ids_grupoCats[index]
    ids_valores_cats=definirGrupoIdValorCategoria(row)
    
    #Poblar dimensión bridge categoria
    for i in ids_valores_cats:
        bridgeCategoria.loc[len(bridgeCategoria)]=[
            grupo_categoria,
            i
        ]

    #id tema interes
    row_tema1=temaInteres[temaInteres['nombre']==row['id-TEMA DE INTERES1']]
    row_tema2=temaInteres[temaInteres['nombre']==row['ID-TEMA DE INTERES2']]
    grupoTemaInteres=ids_grupoTemas[index]

    if not row_tema1.empty:
        bridgeTemaInteres.loc[len(bridgeTemaInteres)]=[
            grupoTemaInteres,
            str(row_tema1['id'].iloc[0])
        ]

    if not row_tema2.empty:
        bridgeTemaInteres.loc[len(bridgeTemaInteres)]=[
            grupoTemaInteres,
            str(row_tema2['id'].iloc[0])
        ]
  
    
    #id fecha modificación y registro
    fecha='1,1,2025'

    #grupo variable longitudinal
    row_vl=bridgeVaribleLongitudinal[bridgeVaribleLongitudinal['idVariableCompuesta']==var]
    if not row_vl.empty:
        id_longitudinal= row_vl.iloc[0]['idGrupoVariable']
    else:
        id_longitudinal= None

    id_operacion=None 
    
    hechoEvaluarVariableSujeto.loc[index]=[
        var,
        id_etapa,
        grupo_categoria,
        grupoTemaInteres,
        fecha,
        fecha,
        id_operacion,
        id_longitudinal
    ]

hechoEvaluarVariableSujeto


,idVariable,idEtapa,idGrupoValorCategoria,idGrupoTemaInteres,idFechaModificacion,idFechaRegistro,idGrupoOperacion,idGrupoVariableLongitudinal
0,663c523f-8520-4a1a-9812-54576497aa1a,e63e00cc-19fc-4d3a-bbdb-84f34766786c,238ee9fe-e436-47f1-a37c-bda9fb621ec3,8adb6a02-992c-4298-ac6c-ced7e0e4e94d,"1,1,2025","1,1,2025",None,None
1,ea6852d6-87ad-46ef-a9ff-edf64bd3e146,e63e00cc-19fc-4d3a-bbdb-84f34766786c,6f1fc303-cb51-43ee-9092-34363f837d52,d8af7f30-928d-4ba9-ade8-51c05df1f9e6,"1,1,2025","1,1,2025",None,None
2,abef5f2e-af99-4ff8-ae67-13c04937edf6,e63e00cc-19fc-4d3a-bbdb-84f34766786c,e8c7c6e3-2ae7-4bc0-85ce-5c00c92a16e9,ec6048b4-324f-4318-a174-bb4702ce2a2f,"1,1,2025","1,1,2025",None,None
3,07a2e42d-34c6-484c-8633-f5323d84ff88,e63e00cc-19fc-4d3a-bbdb-84f34766786c,20263764-643e-4f7a-bc95-4f5779dc19e6,c048bf77-1055-452e-bb86-8f4bd0eb1c92,"1,1,2025","1,1,2025",None,None
4,6694767b-3488-4207-be96-358a46072cc6,e63e00cc-19fc-4d3a-bbdb-84f34766786c,9c5f3097-f257-42a5-b8f4-ed27f58427a8,ef4d0105-8bd1-467e-97e5-0b923c6b637d,"1,1,2025","1,1,2025",None,None
...,...,...,...,...,...,...,...,...
629,4df2bf31-b1c4-4849-b4a4-8dc3868e4efc,d181d06b-68da-408b-89ad-099b6c91c3f6,fc08522b-4bcd-41ae-9ac4-fcdea2c4723f,45472529-4c53-4521-bfa9-6eb692fac4c9,"1,1,2025","1,1,2025",None,None
630,688adc8b-07b3-4b53-a3ea-8e2b801d410a,d181d06b-68da-408b-89ad-099b6c91c3f6,113cd91d-9c87-4c80-a646-2716c0cdd5b9,eb9aafe9-ca6c-44c5-8dc2-a54ac0626f74,"1,1,2025","1,1,2025",None,None
631,6f264a62-fd4b-47dc-99f6-954e465a2207,d181d06b-68da-408b-89ad-099b6c91c3f6,4758930c-bf9f-411a-903b-6d863d045bcd,ec7cdfdd-95c7-4e0e-856d-9bbfa7ccc4b3,"1,1,2025","1,1,2025",None,None
632,89ab22e4-6c05-4b45-b535-cb9754ae118a,d181d06b-68da-408b-89ad-099b6c91c3f6,955a50c4-d27a-47f9-be25-d81ec641fb1a,dc70c901-0981-414e-a6f8-d52644571595,"1,1,2025","1,1,2025",None,None


Anotaciones:
- La meta de donde sale?
- Los eventos no relacionados a la variable pero no a una etapa donde estan?
- En este momento se registraron las operaciones que se hicieron sobre las variables?
- ¿Cómo identificar variables longitudinales?
- Aun no tengo la BD  de indicadores
- Es necesario tener un grupo de temas de interés? (no hay ninguna variable en el diccionario que tenga más de un tema de interés)